# requirimentos

In [ ]:
!pip install gcsfs datasets
!pip install --upgrade sentence-transformers
!pip install --upgrade transformers
!pip install unidecode
!python -m spacy download pt_core_news_md
!pip install --upgrade torch


# treino

In [ ]:
from datasets import Dataset
import pandas as pd
import torch
from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, f1_score

# Fixar semente para reprodução
torch.manual_seed(42)

# Carregar os dados
csv_path = "/content/dataset_transformado_not_lema.csv"  # Atualize com o caminho correto
data = pd.read_csv(csv_path)

# Codificar as classes
label_encoder = LabelEncoder()
data["Label"] = label_encoder.fit_transform(data["Label"])

# Salvar o mapeamento label -> id no modelo
id2label = {i: label for i, label in enumerate(label_encoder.classes_)}
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}

# Calcular pesos das classes
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=data["Label"].unique(),
    y=data["Label"]
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Dividir os dados em treino e teste
train_texts, eval_texts, train_labels, eval_labels = train_test_split(
    data["Texto"], data["Label"], test_size=0.1, stratify=data["Label"], random_state=42
)

# Tokenizar os textos
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased")

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(list(eval_texts), truncation=True, padding=True, max_length=128)

# Criar datasets compatíveis com o Trainer
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_dataset = ClassificationDataset(train_encodings, list(train_labels))
eval_dataset = ClassificationDataset(eval_encodings, list(eval_labels))

# Carregar o modelo para classificação
model = BertForSequenceClassification.from_pretrained(
    "neuralmind/bert-large-portuguese-cased",
    num_labels=len(label_encoder.classes_),  # Número de classes
)

# Adicionar label2id e id2label no config do modelo
model.config.label2id = label2id
model.config.id2label = id2label

# Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="models/bert-ptbr-classification",
    num_train_epochs=5,  # Aumentado para explorar mais o treinamento
    per_device_train_batch_size=32,  # Reduzido para evitar problemas de memória
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,  # Taxa de aprendizado ajustada
    weight_decay=0.01,
    logging_dir="/content/logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=397,
    save_steps=397,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    fp16=True,
    run_name="bert-ptbr-classification",
)

# Definir a métrica de avaliação
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1).numpy()
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

# Criar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Adicionado callback para early stopping
)

# Treinar o modelo
trainer.train()

# Salvar o modelo treinado
output_dir = f"models/bert-ptbr-classification-final"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


# Testes 

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Diretório onde o modelo foi salvo
output_dir = "/content/models/bert-ptbr-classification-final"

# Carregar o modelo e o tokenizer
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Frases para teste
test_phrases = [
    "quero uma honda civic"
]

# Tokenizar as frases de teste
inputs = tokenizer(test_phrases, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Fazer a predição
model.eval()  # Colocar o modelo em modo de avaliação
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy()

# Decodificar os índices das classes previstas usando id2label
predicted_classes = [model.config.id2label[pred] for pred in predictions]

# Exibir as frases e suas respectivas previsões
for phrase, pred_class in zip(test_phrases, predicted_classes):
    print(f"Frase: '{phrase}' -> Classe prevista: '{pred_class}'")


O gato subiu na árvore e miou alto.



In [ ]:
probabilities = torch.softmax(logits, dim=1)

# Obter as 10 maiores probabilidades e seus índices
values, indices = torch.topk(probabilities, 30, dim=1)
rounded_values = [round(value.item(), 2) for value in values[0]]

# Usar o id2label para mapear índices para rótulos
id2label = model.config.id2label
top_labels = [id2label[idx.item()] for idx in indices[0]]

print("Probabilidades dos 10 maiores:", rounded_values)
print("Índices dos 10 maiores:", top_labels)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Preparar os dados de teste para avaliação
test_texts, test_labels = eval_texts.tolist(), eval_labels.tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)
test_dataset = ClassificationDataset(test_encodings, test_labels)

# Realizar predições no conjunto de teste
predictions = trainer.predict(test_dataset)
pred_logits = predictions.predictions
pred_labels = np.argmax(pred_logits, axis=1)

# Calcular a matriz de confusão e o relatório de classificação
conf_matrix = confusion_matrix(test_labels, pred_labels)
class_report = classification_report(test_labels, pred_labels, target_names=label_encoder.classes_)

# Exibir a matriz de confusão
print("Matriz de Confusão:")
print(conf_matrix)

# Exibir o relatório de classificação
print("\nRelatório de Classificação (Taxa de Acerto por Classe):")
print(class_report)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

# Calcular a matriz de confusão
conf_matrix = confusion_matrix(test_labels, pred_labels)

# Calcular a precisão do modelo
accuracy = accuracy_score(test_labels, pred_labels)

# Plotar a matriz de confusão com a precisão no título
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title(f"Confusion Matrix\nAccuracy: {accuracy:.2%}")
plt.show()

# Salvar a matriz de confusão como arquivo
plt.savefig("confusion_matrix.png")


# Upload

In [ ]:
print('iniciando ..')
from transformers import BertForSequenceClassification, BertTokenizer
from huggingface_hub import login


#login()

# Caminho do modelo local
output_dir = "/content/models/bert-ptbr-classification-final"

# Adicionar mapeamento de labels
model.config.label2id = label2id
model.config.id2label = id2label

# Salvar modelo e tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Subir para o Hugging Face Hub
model.push_to_hub("SenhorDasMoscas/acho-classification-14-01-2025")
tokenizer.push_to_hub("SenhorDasMoscas/acho-classification-14-01-2025")

# Carregar do Hugging Face Hub
model = BertForSequenceClassification.from_pretrained("SenhorDasMoscas/acho-classification-14-01-2025")
tokenizer = BertTokenizer.from_pretrained("SenhorDasMoscas/acho-classification-14-01-2025")

# Testar
test_phrases = ["quero ajeitar o ar do meu carro"]
inputs = tokenizer(test_phrases, return_tensors="pt", truncation=True, padding=True, max_length=128)
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1).numpy()

# Decodificar
predicted_classes = [model.config.id2label[pred] for pred in predictions]
for phrase, pred_class in zip(test_phrases, predicted_classes):
    print(f"Frase: '{phrase}' -> Classe prevista: '{pred_class}'")
